In [24]:
import wbdata
import pandas as pd
import datetime

# Define the time period
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Define the major economies (G20 countries)
major_economies = [
    "USA", "CAN", "MEX", "BRA", "ARG",
    "DEU", "GBR", "FRA", "ITA", "EUU",
    "RUS", "TUR", "SAU", "ZAF", "CHN",
    "IND", "IDN", "JPN", "KOR", "AUS"
]

# Define indicators for GDP, inflation, unemployment, and exchange rates
indicators = {
    'NY.GDP.MKTP.CD': 'GDP',         # GDP (current US$)
    'FP.CPI.TOTL.ZG': 'Inflation',   # Inflation, consumer prices (annual %)
    'SL.UEM.TOTL.ZS': 'Unemployment',# Unemployment, total (% of total labor force)
    'PA.NUS.FCRF': 'Exchange_Rate'   # Official exchange rate (LCU per US$, period average)
}

# Download data for the defined indicators
try:
    # Retrieve economic data
    economic_data = wbdata.get_dataframe(indicators, country=major_economies)
    
    # Reset index to access 'date' column
    economic_data.reset_index(inplace=True)
    
    # Convert 'date' column to datetime format
    economic_data['date'] = pd.to_datetime(economic_data['date'])
    
    # Filter data based on start_date and end_date
    economic_data_filtered = economic_data[(economic_data['date'] >= start_date) & (economic_data['date'] <= end_date)]
    
    # Pivot the data
    economic_data_pivot = economic_data_filtered.pivot_table(index='date', columns='country', values=indicators.values())
    economic_data_pivot = economic_data_pivot.fillna(method='ffill').dropna(how='all')
    
    # Save each indicator to a separate CSV file
    for indicator in indicators.values():
        csv_file_path = f'economic_data_{indicator.lower()}_pivot.csv'
        economic_data_pivot[indicator].to_csv(csv_file_path, header=True)
        print(f"Successfully saved pivoted {indicator} data to {csv_file_path}")

except Exception as e:
    print(f"An error occurred while downloading or saving economic data: {e}")

Successfully saved pivoted GDP data to economic_data_gdp_pivot.csv
Successfully saved pivoted Inflation data to economic_data_inflation_pivot.csv
Successfully saved pivoted Unemployment data to economic_data_unemployment_pivot.csv
Successfully saved pivoted Exchange_Rate data to economic_data_exchange_rate_pivot.csv


C:\Users\User\AppData\Local\Temp\ipykernel_6592\4049391382.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  economic_data_pivot = economic_data_pivot.fillna(method='ffill').dropna(how='all')


In [32]:
import wbdata
import pandas as pd
import datetime

# Define the time period
start_date = datetime.datetime(1987, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Define the world as the country
world = ["WLD"]

# Define indicators for GDP, inflation, and unemployment
indicators = {
    'NY.GDP.MKTP.CD': 'GDP',              # GDP (current US$)
    'FP.CPI.TOTL.ZG': 'Inflation',        # Inflation, consumer prices (annual %)
    'SL.UEM.TOTL.ZS': 'Unemployment'      # Unemployment, total (% of total labor force)
}

# Download data for the defined indicators
try:
    # Fetch annual data without the convert_date argument
    economic_data = wbdata.get_dataframe(indicators, country=world)
    economic_data.reset_index(inplace=True)

    # Convert 'date' column to datetime format manually
    economic_data['date'] = pd.to_datetime(economic_data['date'].astype(str) + '-12-31')
    
    # Filter data based on start_date and end_date
    economic_data_filtered = economic_data[(economic_data['date'] >= start_date) & (economic_data['date'] <= end_date)]
    
    # Select relevant columns
    economic_data_filtered = economic_data_filtered[['date', 'GDP', 'Inflation', 'Unemployment']].dropna()

    # Sort data by date in ascending order
    economic_data_filtered = economic_data_filtered.sort_values(by='date', ascending=True)

    # Set date as index for resampling
    economic_data_filtered.set_index('date', inplace=True)

    # Resample to monthly frequency and interpolate missing values
    monthly_data = economic_data_filtered.resample('M').interpolate(method='linear')

    # Save to CSV
    monthly_data.to_csv('world_economic_data.csv', index=True)
    
    # Check data sample
    print("Economic Data Sample:")
    print(monthly_data.head(10))

except Exception as e:
    print(f"An error occurred while downloading economic data: {e}")

Economic Data Sample:
                     GDP  Inflation  Unemployment
date                                             
1991-12-31  2.382355e+13   8.996939      4.996340
1992-01-31  2.396059e+13   8.883536      5.009140
1992-02-29  2.409763e+13   8.770134      5.021940
1992-03-31  2.423467e+13   8.656731      5.034740
1992-04-30  2.437171e+13   8.543329      5.047540
1992-05-31  2.450874e+13   8.429926      5.060340
1992-06-30  2.464578e+13   8.316524      5.073139
1992-07-31  2.478282e+13   8.203121      5.085939
1992-08-31  2.491986e+13   8.089719      5.098739
1992-09-30  2.505690e+13   7.976316      5.111539
